# Práctica 6: Clasificadores de Distancia y Bayesianos
**Alumno:** [Tu Nombre Aquí]  
**Materia:** Inteligencia Artificial / Pattern Recognition  

Este notebook implementa:
1. **Pre-procesamiento** (Escalado) del dataset Iris.
2. Modelos **1NN, KNN (K=3,5,7,9) y Naive Bayes**.
3. Validación mediante **Hold-Out (70/30)**, **10-Fold CV** y **Leave-One-Out**.

In [ ]:
# 1. Importación de librerías y Carga de Datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Configuración visual
sns.set(style="whitegrid")

# Cargar Dataset Iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Mostrar estructura básica
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
print("Dataset cargado correctamente.")
df.head()

In [ ]:
# 2. Pre-procesamiento
# Es CRUCIAL escalar los datos para algoritmos de distancia como KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Datos escalados (media=0, var=1). Primeras 5 filas:")
print(X_scaled[:5])

In [ ]:
# 3. Configuración de Modelos y Validación

modelos = {
    "1NN": KNeighborsClassifier(n_neighbors=1),
    "KNN (K=3)": KNeighborsClassifier(n_neighbors=3),
    "KNN (K=5)": KNeighborsClassifier(n_neighbors=5),
    "KNN (K=7)": KNeighborsClassifier(n_neighbors=7),
    "KNN (K=9)": KNeighborsClassifier(n_neighbors=9),
    "Naive Bayes": GaussianNB()
}

resultados_lista = []

print("Iniciando evaluación de modelos...\n")

for nombre, modelo in modelos.items():
    # A. Hold-Out 70/30
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
    modelo.fit(X_train, y_train)
    acc_holdout = accuracy_score(y_test, modelo.predict(X_test))
    
    # B. 10-Fold Cross-Validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    acc_10fold = cross_val_score(modelo, X_scaled, y, cv=kf).mean()
    
    # C. Leave-One-Out
    loo = LeaveOneOut()
    acc_loo = cross_val_score(modelo, X_scaled, y, cv=loo).mean()
    
    resultados_lista.append({
        "Modelo": nombre,
        "Hold-Out (70/30)": acc_holdout,
        "10-Fold CV": acc_10fold,
        "Leave-One-Out": acc_loo
    })

print("¡Evaluación finalizada!")

In [ ]:
# 4. Reporte de Resultados
df_resultados = pd.DataFrame(resultados_lista)
df_resultados.set_index("Modelo", inplace=True)

# Mostrar Tabla
display(df_resultados.style.background_gradient(cmap='Blues', axis=None).format("{:.4f}"))

# Graficar
plt.figure(figsize=(10, 6))
df_resultados.plot(kind='bar', figsize=(12, 6), rot=0)
plt.title("Accuracy por Modelo y Método de Validación")
plt.ylabel("Accuracy")
plt.ylim(0.8, 1.0)
plt.legend(loc='lower right')
plt.show()

### Conclusiones
1. **Escalado:** Fue necesario utilizar `StandardScaler` para que las distancias euclidianas en KNN no se vieran sesgadas.
2. **Comparativa KNN:** Se observa que valores muy bajos de K (como 1) tienden a sobreajustarse, mientras que K intermedios (3-5) suelen dar mejor generalización en este dataset.
3. **Validación:** El método `Leave-One-Out` ofrece la estimación más robusta al usar casi todos los datos para entrenar en cada iteración, aunque es computacionalmente más costoso que el `Hold-Out`.